# साधारण छवि वर्गीकरणकर्ता

यो नोटबुकले तपाईंलाई पूर्व-प्रशिक्षित न्यूरल नेटवर्क प्रयोग गरेर छविहरू वर्गीकृत गर्ने तरिका देखाउँछ।

**तपाईंले के सिक्नुहुनेछ:**
- पूर्व-प्रशिक्षित मोडेल कसरी लोड गर्ने र प्रयोग गर्ने
- छवि पूर्व-प्रक्रिया
- छविहरूमा भविष्यवाणी गर्ने तरिका
- आत्मविश्वास स्कोर बुझ्ने

**प्रयोगको मामला:** छविहरूमा वस्तुहरू पहिचान गर्नुहोस् (जस्तै "बिरालो", "कुकुर", "कार", आदि) 

---


## चरण १: आवश्यक लाइब्रेरीहरू आयात गर्नुहोस्

हामीलाई चाहिने उपकरणहरू आयात गरौं। यदि तपाईंलाई यी सबै बुझ्न गाह्रो भयो भने चिन्ता नगर्नुहोस्!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## चरण २: प्रि-ट्रेन गरिएको मोडेल लोड गर्नुहोस्

हामी **MobileNetV2** प्रयोग गर्नेछौं, जुन लाखौं तस्बिरहरूमा पहिले नै प्रशिक्षित गरिएको न्युरल नेटवर्क हो।

यसलाई **ट्रान्सफर लर्निङ** भनिन्छ - अरूले प्रशिक्षित गरेको मोडेल प्रयोग गर्नु!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## चरण ३: सहायक फङ्सनहरू

हामी हाम्रो मोडेलका लागि छविहरू लोड गर्न र तयार गर्न फङ्सनहरू बनाउँछौं।


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## चरण ४: नमूना छविहरूमा परीक्षण गर्नुहोस्

इन्टरनेटबाट केही छविहरू वर्गीकरण गर्ने प्रयास गरौं!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### प्रत्येक छविको वर्गीकरण गर्नुहोस्


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## चरण ५: आफ्नै तस्वीरहरू प्रयास गर्नुहोस्!

तलको URL लाई तपाईं वर्गीकरण गर्न चाहनुभएको कुनै पनि तस्वीरको URL सँग बदल्नुहोस्।


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 के भयो?

1. **हामीले प्रि-ट्रेन गरिएको मोडेल लोड गर्यौं** - MobileNetV2 लाखौं तस्बिरहरूमा प्रशिक्षित गरिएको थियो  
2. **हामीले तस्बिरहरू प्रि-प्रोसेस गर्यौं** - मोडेलका लागि आकार परिवर्तन गरियो र फर्म्याट गरियो  
3. **मोडेलले भविष्यवाणी गर्यो** - यसले 1000 वस्तु वर्गहरूको लागि सम्भावनाहरू उत्पादन गर्यो  
4. **हामीले नतिजा डिकोड गर्यौं** - संख्याहरूलाई मानवले बुझ्न सकिने लेबलमा परिवर्तन गरियो  

### विश्वास स्कोर बुझ्दै

- **९०-१००%**: धेरै विश्वस्त (लगभग पक्का सही)  
- **७०-९०%**: विश्वस्त (सायद सही)  
- **५०-७०%**: केही हदसम्म विश्वस्त (सही हुन सक्छ)  
- **५०% भन्दा कम**: धेरै विश्वस्त छैन (अनिश्चित)  

### किन भविष्यवाणीहरू गलत हुन सक्छन्?

- **असामान्य कोण वा प्रकाश** - मोडेल सामान्य तस्बिरहरूमा प्रशिक्षित गरिएको थियो  
- **धेरै वस्तुहरू** - मोडेलले एक मुख्य वस्तु अपेक्षा गर्छ  
- **दुर्लभ वस्तुहरू** - मोडेलले केवल 1000 वर्गहरू मात्र चिन्छ  
- **कम गुणस्तरको तस्बिर** - धमिलो वा पिक्सेलेटेड तस्बिरहरू कठिन हुन्छन्  

---  


## 🚀 आगामी कदमहरू

1. **विभिन्न तस्बिरहरू प्रयास गर्नुहोस्:**
   - [Unsplash](https://unsplash.com) मा तस्बिरहरू खोज्नुहोस्
   - दायाँ-क्लिक गर्नुहोस् → "Copy image address" गरेर URL प्राप्त गर्नुहोस्

2. **परीक्षण गर्नुहोस्:**
   - अमूर्त कलासँग के हुन्छ?
   - के यसले विभिन्न कोणबाट वस्तुहरू चिन्न सक्छ?
   - यसले धेरै वस्तुहरूलाई कसरी व्यवस्थापन गर्छ?

3. **थप जान्नुहोस्:**
   - [Computer Vision पाठहरू](../lessons/4-ComputerVision/README.md) अन्वेषण गर्नुहोस्
   - आफ्नो तस्बिर वर्गीकरणकर्ता प्रशिक्षण गर्न सिक्नुहोस्
   - CNNs (Convolutional Neural Networks) कसरी काम गर्छन् बुझ्नुहोस्

---

## 🎉 बधाई छ!

तपाईंले अत्याधुनिक न्यूरल नेटवर्क प्रयोग गरेर तस्बिर वर्गीकरणकर्ता निर्माण गर्नुभयो!

यही प्रविधिले निम्नलाई शक्ति दिन्छ:
- Google Photos (तपाईंका तस्बिरहरू व्यवस्थित गर्दै)
- स्वचालित कारहरू (वस्तुहरू चिन्दै)
- चिकित्सा निदान (X-rays विश्लेषण गर्दै)
- गुणस्तर नियन्त्रण (त्रुटिहरू पत्ता लगाउँदै)

अन्वेषण र सिकाइ जारी राख्नुहोस्! 🚀



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
